## Plotly Dashboard

In [74]:
!pip install dash==1.19.0  

!pip install jupyter_dash 

!pip install --upgrade plotly

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 1.0/1.0 MB 7.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ------------------ --------------------- 1.6/3.5 MB 8.1 MB/s eta 0:00:01
     ------------------------------------ 


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
   --- ------------------------------------ 1.6/16.3 MB 9.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.9/16.3 MB 7.7 MB/s eta 0:00:02
   ---------- ----------------------------- 4.5/16.3 MB 7.5 MB/s eta 0:00:02
   ------------- -------------------------- 5.5/16.3 MB 6.9 MB/s eta 0:00:02
   ------------------ --------------------- 7.3/16.3 MB 7.1 MB/s eta 0:00:02
   ------------------- -------------------- 8.1/16.3 MB 6.6 MB/s eta 0:00:02
   ---------------------- ----------------- 9.2/16.3 MB 6.7 MB/s eta 0:00:02
   ------------------------- -------------- 10.2/16.3 MB 6.3 MB/s eta 0:00:01
   ----------------------------- ---------- 12.1/16.3 MB 6.5 MB/s eta 0:00:01
   --------------------------------- ------ 13.6/16.3 MB 6.6 MB/s eta 0:00:01
   ------------------------------------ --- 14.9/16.3 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------  16.3/16.3 MB 6.6 MB/s eta 0:00:01
 


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
#pip install --upgrade dash flask-compress plotly

In [24]:
#pip install --upgrade werkzeug

In [18]:
# Import necessary libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [22]:
# Read the SpaceX launch data into a pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Get min and max payload values for slider
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a Dash application
app = dash.Dash(__name__)

# App layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Dropdown for launch site selection
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                 ],
                 value='ALL',
                 placeholder="Select a launch site",
                 searchable=True),
    html.Br(),

    # Pie chart for launch success counts
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # Payload range slider
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[min_payload, max_payload]),

    # Scatter plot for payload and success correlation
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback for pie chart
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
                     names='Launch Site',
                     title='Total Success Launches By Sites')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        success_count = filtered_df.groupby('class').count().reset_index()
        fig = px.pie(success_count, values='Launch Site', 
                     names='class', 
                     title=f'Total Success Launches for {entered_site}')
    return fig

# Callback for scatter plot
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) &
                            (spacex_df['Payload Mass (kg)'] <= payload[1])]

    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Correlation between Payload and Success for All Sites')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Correlation between Payload and Success for {entered_site}')
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run(debug=True, port=8087)